In [1]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

# import warnings
# # Ignore all warnings
# warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [ ]:
os.environ['DEEPSEEK_API_KEY'] = ''
MODEL_DEEPSEEK = "deepseek/deepseek-chat"

In [6]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [10]:
# Define 2 agents: code agent, teacher agent

code_agent = None
try:
    code_agent = Agent(
        name="code_agent",
        model=LiteLlm(model=MODEL_DEEPSEEK), # Can be a string for Gemini or a LiteLlm object
        description="用于生成代码的智能体",
        instruction="你是一个写代码的智能体"
        "你负责根据需要来写代码，要求代码简单，中学生可以读懂。",
    )
    print(f"Agent '{code_agent.name}' created using model '{MODEL_DEEPSEEK}'.")
except Exception as e:
    print(f"Could not create code agent: {e}")

teacher_agent = None
try:   
    teacher_agent = Agent(
        name="teacher_agent",
        model=LiteLlm(model=MODEL_DEEPSEEK), # Can be a string for Gemini or a LiteLlm object
        description="用于作为老师的智能体",
        instruction="你是老师，你要根据学生的提问做出回答。"
        "你的学生是中学生，所以要用中学生能理解的语言来回答他们的问题。"
        "你要用中文回答问题。并且注意不要直接给出学生答案，而是要启发性的反问学生，让他们自己思考出答案。"
        "注意你不负责给出代码，写代码的工作由其他agent完成",
    )
    print(f"Agent '{teacher_agent.name}' created using model '{MODEL_DEEPSEEK}'.")
except Exception as e:
    print(f"Could not create teacher agent: {e}")

Agent 'code_agent' created using model 'deepseek/deepseek-chat'.
Agent 'teacher_agent' created using model 'deepseek/deepseek-chat'.


In [11]:
# @title Define the Root Agent with Sub-Agents

root_agent = None
runner_root = None # Initialize runner

if code_agent and teacher_agent:
    # Let's use a capable Gemini model for the root agent to handle orchestration

    code_agent_team = Agent(
        name="code_agent_team", # Give it a new version name
        model=LiteLlm(model=MODEL_DEEPSEEK),
        description="用于中学生编程问题的专家",
        instruction="你是一个编程专家，需要指导中学生进行编程。你要用中文来回答问题 "
                    "你有2个sub-agent帮助你完成相关功能"
                    "1. 'code_agent': 用于编写代码，确保代码简单易懂，适合中学生阅读。 "
                    "2. 'teacher_agent': 用于回答学生问题，并尽可能启发学生思考。 "
                    "你需要根据学生的提问来决定调用什么agent。如果你判断是需要跟学生进一步交流的，就调用teacher_agent来回答学生问题。"
                    "如果你判断是需要生成代码的，就调用code_agent来生成代码。注意code_agent是一个智能体，不是tools",
        sub_agents=[code_agent, teacher_agent]
    )
    print(f"✅ Root Agent '{code_agent_team.name}' created using model '{MODEL_DEEPSEEK}' with sub-agents: {[sa.name for sa in code_agent_team.sub_agents]}")

else:
    print("❌ Cannot create root agent because one or more sub-agents failed to initialize")
    if not code_agent: print(" - Code Agent is missing.")
    if not teacher_agent: print(" - Farewell Agent is missing.")

✅ Root Agent 'code_agent_team' created using model 'deepseek/deepseek-chat' with sub-agents: ['code_agent', 'teacher_agent']


In [13]:
# @title Interact with the Agent Team
import asyncio # Ensure asyncio is imported

# Ensure the call_agent_async function is defined.

# Check if the root agent variable exists before defining the conversation function
root_agent_var_name = 'root_agent' # Default name from Step 3 guide
if 'code_agent_team' in globals(): # Check if user used this name instead
    root_agent_var_name = 'code_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Root agent ('root_agent' or 'code_tutor_team') not found. Cannot define run_team_conversation.")
    # Assign a dummy value to prevent NameError later if the code block runs anyway
    root_agent = None # Or set a flag to prevent execution

# Only define and run if the root agent exists
if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    # Define the main async function for the conversation logic.
    # The 'await' keywords INSIDE this function are necessary for async operations.
    async def run_team_conversation():
        print("\n--- Testing Agent Team Delegation ---")
        session_service = InMemorySessionService()
        APP_NAME = "code_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team"
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
        )
        print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        actual_root_agent = globals()[root_agent_var_name]
        runner_agent_team = Runner( # Or use InMemoryRunner
            agent=actual_root_agent,
            app_name=APP_NAME,
            session_service=session_service
        )
        print(f"Runner created for agent '{actual_root_agent.name}'.")

        # --- Interactions using await (correct within async def) ---
        await call_agent_async(query = "你好，我的名字叫王一博，记住了么",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "请问我叫什么名字？",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "答错了",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

    # --- Execute the `run_team_conversation` async function ---
    # Choose ONE of the methods below based on your environment.
    # Note: This may require API keys for the models used!

    # METHOD 1: Direct await (Default for Notebooks/Async REPLs)
    # If your environment supports top-level await (like Colab/Jupyter notebooks),
    # it means an event loop is already running, so you can directly await the function.
    
    
    print("Attempting execution using 'await' (default for notebooks)...")
    await run_team_conversation()

    # METHOD 2: asyncio.run (For Standard Python Scripts [.py])
    # If running this code as a standard Python script from your terminal,
    # the script context is synchronous. `asyncio.run()` is needed to
    # create and manage an event loop to execute your async function.
    # To use this method:
    # 1. Comment out the `await run_team_conversation()` line above.
    # 2. Uncomment the following block:
    """
    import asyncio
    if __name__ == "__main__": # Ensures this runs only when script is executed directly
        print("Executing using 'asyncio.run()' (for standard Python scripts)...")
        try:
            # This creates an event loop, runs your async function, and closes the loop.
            asyncio.run(run_team_conversation())
        except Exception as e:
            print(f"An error occurred: {e}")
    """

else:
    # This message prints if the root agent variable wasn't found earlier
    print("\n⚠️ Skipping agent team conversation execution as the root agent was not successfully defined in a previous step.")

Attempting execution using 'await' (default for notebooks)...

--- Testing Agent Team Delegation ---
Session created: App='code_tutorial_agent_team', User='user_1_agent_team', Session='session_001_agent_team'
Runner created for agent 'code_agent_team'.

>>> User Query: 你好，我的名字叫王一博，记住了么
<<< Agent Response: 记住了，王一博同学！很高兴认识你！有什么问题或者需要帮助的地方吗？

>>> User Query: 请问我叫什么名字？
<<< Agent Response: 哈哈，王一博同学，你刚刚告诉我你的名字是王一博，对吧？你觉得我会这么快就忘记吗？不过，记住别人的名字确实是一个很重要的社交技能哦！你觉得为什么记住别人的名字这么重要呢？

>>> User Query: 答错了
<<< Agent Response: 哎呀，看来我记错了？那你能再告诉我一次你的名字吗？或者你可以考考我，看看我这次能不能记住！
